In [ ]:
import os
import sys

sys.path.append("../")

import torch

from datasets.fastspeech_dataset import (
    build_path_to_transcript_dict_libri_tts,
    FastSpeechDataset, Miniset
)
from pipelines.gst_fastspeech2 import meta_train_loop

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
DATASET_BASE_PATH = '../data/librispeech'
# TEST_CLEAN_PATH = os.path.join(DATASET_BASE_PATH, 'test-clean')
TEST_CLEAN_PATH = '/home/nmmy/Documents/dataset/test-clean/'
TRAIN_CLEAN_PATH = os.path.join(DATASET_BASE_PATH, 'train-clean-100')
ALIGNER_CHECKPOINT = "/home/nmmy/Documents/dataset/aligner.pt"

In [ ]:
transcript_dict = build_path_to_transcript_dict_libri_tts(TEST_CLEAN_PATH)

In [ ]:
# only test on 100 samples
transcript_dict_100 = {}
for i,(k,v) in enumerate(transcript_dict.items()):
    if i == 100: break
    transcript_dict_100[k] = v

In [ ]:
dataset = FastSpeechDataset(
    path_to_transcript_dict=transcript_dict,
    acoustic_checkpoint_path=ALIGNER_CHECKPOINT,  # path to aligner.pt
    cache_dir="./librispeech",
    lang="en",
    loading_processes=4,  # depended on how many CPU you have
    device=device,
)

In [ ]:
spkId = os.path.join(TEST_CLEAN_PATH, '61')

In [ ]:
minidataset = Miniset(dataset, speakerID='61')

In [ ]:
datasetlist = [minidataset]

In [ ]:
from tts.models.fastspeech2.FastSpeech2 import FastSpeech2

In [ ]:
net = FastSpeech2()

In [ ]:
meta_train_loop.train_loop(net,datasetlist, 
                           device='cpu', 
                           batch_size=2,
                           save_directory='.',
                           path_to_checkpoint=None,
                           phase_1_steps=2,
                           phase_2_steps=2,
                           steps_per_checkpoint=1,
                           path_to_embed_model='/home/nmmy/Documents/dataset/embedding_function.pt',
                           lr=0.01)